# Kvikio experiment

To get this to work we need
1. https://github.com/zarr-developers/zarr-python/pull/934
2. https://github.com/dcherian/xarray/tree/kvikio

In [14]:
%load_ext watermark

# These imports are currently unnecessary.
# cupy_xarray registers the kvikio entrypoint on install.
#import cupy as cp
#import cupy_xarray
#import kvikio.zarr

import numpy as np
import xarray as xr
import zarr

store = "./air-temperature.zarr"

%watermark -iv

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
zarr  : 2.12.1.dev38
sys   : 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 06:04:10) 
[GCC 10.3.0]
numpy : 1.23.1
json  : 2.0.9
xarray: 2022.6.1.dev7+g3f7cc2da3
pandas: 1.4.3



## Create example dataset

at the momment this needs xarray released version + pooch + netCDF4

- Something is broken on the dcherian/kvikio branch
- cannot be compressed
- must read with consolidated=False

In [ ]:
airt = xr.tutorial.open_dataset("air_temperature", engine="netcdf4")
for var in airt.variables:
    airt[var].encoding["compressor"] = None
airt.to_zarr(store, mode="w", consolidated=False)

## Test opening

### Standard usage

In [3]:
xr.open_dataset(store).air

/glade/u/home/dcherian/python/xarray/xarray/backends/plugins.py:117: RuntimeWarning: 'netcdf4' fails while guessing
  warnings.warn(f"{engine!r} fails while guessing", RuntimeWarning)
/glade/scratch/dcherian/tmp/ipykernel_172003/3542870433.py:1: RuntimeWarning: Failed to open Zarr store with consolidated metadata, falling back to try reading non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  xr.open_dataset(store).air


<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
Dimensions without coordinates: time, lon
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

### Now with kvikio!

In [4]:
# Consolidated must be False
ds = xr.open_dataset(store, engine="kvikio", consolidated=False)
ds

<xarray.Dataset>
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
Dimensions without coordinates: time, lon
Data variables:
    air      (time, lat, lon) float32 ...
Attributes:
    Conventions:  COARDS
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...
    title:        4x daily NMC reanalysis (1948)

## Lazy reading

In [5]:
ds.air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
[3869000 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
Dimensions without coordinates: time, lon
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

## Data load for repr

In [6]:
ds["air"].isel(time=0, lat=10)

<xarray.DataArray 'air' (lon: 53)>
array([277.29   , 277.4    , 277.79   , 278.6    , 279.5    , 280.1    ,
       280.6    , 280.9    , 280.79   , 280.69998, 280.79   , 281.     ,
       280.29   , 277.69998, 273.5    , 269.     , 265.5    , 264.     ,
       265.19998, 268.1    , 269.79   , 267.9    , 263.     , 258.1    ,
       254.59999, 251.79999, 249.59999, 249.89   , 252.29999, 254.     ,
       254.29999, 255.89   , 260.     , 263.     , 261.5    , 257.29   ,
       255.5    , 258.29   , 264.     , 268.69998, 270.5    , 270.6    ,
       271.19998, 272.9    , 274.79   , 276.4    , 278.19998, 280.5    ,
       282.9    , 284.69998, 286.1    , 286.9    , 286.6    ], dtype=float32)
Coordinates:
    lat      float32 nan
Dimensions without coordinates: lon
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature

## Load to host

In [11]:
print(type(ds.air.data), type(ds.air.as_numpy().data))
ds.air.as_numpy()

<class 'cupy._core.core.ndarray'> <class 'numpy.ndarray'>


<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)>
array([[[241.2    , 242.5    , 243.5    , ..., 232.79999, 235.5    ,
         238.59999],
        [243.79999, 244.5    , 244.7    , ..., 232.79999, 235.29999,
         239.29999],
        [250.     , 249.79999, 248.89   , ..., 233.2    , 236.39   ,
         241.7    ],
        ...,
        [296.6    , 296.19998, 296.4    , ..., 295.4    , 295.1    ,
         294.69998],
        [295.9    , 296.19998, 296.79   , ..., 295.9    , 295.9    ,
         295.19998],
        [296.29   , 296.79   , 297.1    , ..., 296.9    , 296.79   ,
         296.6    ]],

       [[242.09999, 242.7    , 243.09999, ..., 232.     , 233.59999,
         235.79999],
        [243.59999, 244.09999, 244.2    , ..., 231.     , 232.5    ,
         235.7    ],
        [253.2    , 252.89   , 252.09999, ..., 230.79999, 233.39   ,
         238.5    ],
...
        [293.69   , 293.88998, 295.38998, ..., 295.09   , 294.69   ,
         294.29   ],
        [296.29   , 297.19   , 297.59   , ..., 295.29   , 295.09   ,
         294.38998],
        [297.79   , 298.38998, 298.49   , ..., 295.69   , 295.49   ,
         295.19   ]],

       [[245.09   , 244.29   , 243.29   , ..., 241.68999, 241.48999,
         241.79   ],
        [249.89   , 249.29   , 248.39   , ..., 239.59   , 240.29   ,
         241.68999],
        [262.99   , 262.19   , 261.38998, ..., 239.89   , 242.59   ,
         246.29   ],
        ...,
        [293.79   , 293.69   , 295.09   , ..., 295.29   , 295.09   ,
         294.69   ],
        [296.09   , 296.88998, 297.19   , ..., 295.69   , 295.69   ,
         295.19   ],
        [297.69   , 298.09   , 298.09   , ..., 296.49   , 296.19   ,
         295.69   ]]], dtype=float32)
Coordinates:
  * lat      (lat) float32 nan nan nan nan nan nan ... nan nan nan nan nan nan
Dimensions without coordinates: time, lon
Attributes:
    GRIB_id:       11
    GRIB_name:     TMP
    actual_range:  [185.16000366210938, 322.1000061035156]
    dataset:       NMC Reanalysis
    level_desc:    Surface
    long_name:     4xDaily Air temperature at sigma level 995
    parent_stat:   Other
    precision:     2
    statistic:     Individual Obs
    units:         degK
    var_desc:      Air temperature